In [1]:
import os
import pickle
import numpy as np
import itertools
import random
from fastDamerauLevenshtein import damerauLevenshtein
import csv
import json

## Load datasets and lexicon

In [2]:
def load_dataset(dataset_fpath):
    examples = {}
    with open(dataset_fpath, 'r') as fd:
        reader = csv.reader(fd, delimiter='\t')
        for i, row in enumerate(reader):
            if i == 0: continue
            examples[int(row[0])] = row
    return examples

print('Load validation set... ', end='')
val_examples = load_dataset('data/mimic_synthetic/val.tsv')
print(f'done! {len(val_examples)} examples')

print('Load test set... ', end='')
test_examples = load_dataset('data/mimic_synthetic/test.tsv')
print(f'done! {len(test_examples)} examples')

print('Load clinspell set... ', end='')
clinspell_examples = load_dataset('data/mimic_clinspell_new/test.tsv')
print(f'done! {len(clinspell_examples)} examples')

Load validation set... done! 10000 examples
Load test set... done! 100000 examples
Load clinspell set... done! 873 examples


In [3]:
print('Load lexicon... ', end='')
with open('data/new_vocab/lexicon.json', 'r') as fd:
    dictionary = set(json.load(fd))
print(f'done! {len(dictionary)} words')

Load lexicon... done! 822919 words


## Load results & show correct count

In [4]:
def load_results(result_fpath):
    with open(result_fpath, 'rb') as fd:
        results = pickle.load(fd)
    return results

def get_correct_counts(results):
    correct = len([r for r in results if r['correct'] == r['output'][0]])
    return correct

def acc_string(results_fpath):
    results = load_results(results_fpath)
    total = len(results)
    correct = get_correct_counts(results)   
    return f'{correct:4d}/{total:5d}={correct/total:.4f}'

In [5]:
results_fpath_val_lmeddict = '../exp/cond_indep_typo/mimic_synthetic_12layers6/results_val_beam30_ed5.0_lp1.0_bsFalse_fsTrue.pkl'
results_correct_fpath_test_lmeddict = '../exp/cond_indep_typo/mimic_synthetic_12layers6/correct_val_beam30_ed5.0_lp1.0_bsFalse_fsTrue.pkl'
results_fpath_test_lmeddict = '../exp/cond_indep_typo/mimic_synthetic_12layers6/results_test_beam30_ed5.0_lp1.0_bsFalse_fsTrue.pkl'
results_correct_fpath_test_lmeddict = '../exp/cond_indep_typo/mimic_synthetic_12layers6/correct_test_beam30_ed5.0_lp1.0_bsFalse_fsTrue.pkl'

In [6]:
print(f'Accuracy: {acc_string(results_fpath_test_lmeddict)}')

Accuracy:  679/  873=0.7778


In [7]:
examples = clinspell_examples
resultss = [load_results(results_fpath_test_lmeddict)]
model_names = ['LM+ED+dict']

total_cnt = {}
correct_cnt = {}
typo_correct_dict = {}

for i in range(len(resultss[0])):
    ex_id = resultss[0][i]['example_id']
    example = examples[ex_id]
    typo, left, right, correct = example[-4:]
    outputss = [results[i] for results in resultss]
    
    if typo not in correct_cnt:
        correct_cnt[typo] = [0] * len(model_names)
        typo_correct_dict[typo] = correct
        total_cnt[typo] = 0
    
    total_cnt[typo] += 1
    for i, output in enumerate(outputss):
        if output['output'][0][3:-4] == correct:
            correct_cnt[typo][i] += 1
            
print(' ' * 37 + '(Total) ' + ' '.join(model_names))

frequent_typos = list(zip(*sorted(total_cnt.items(), key=lambda x: x[1], reverse=True)))[0]
for typo in frequent_typos:
    correct = typo_correct_dict[typo]
    print_str = f'{typo}/{correct}'
    print_str += ' ' * (40 - len(print_str))
    print_str += f'({total_cnt[typo]:2d})'
    for cnt in correct_cnt[typo]:
        print_str += f'{cnt:4d}'
    print(print_str)

                                     (Total) LM+ED+dict
lugns/lungs                             ( 5)   5
responsvie/responsive                   ( 5)   5
pylenonephritis/pyelonephritis          ( 5)   1
noteed/noted                            ( 5)   5
hypercholesteromia/hypercholesteremia   ( 5)   0
spenomegaly/splenomegaly                ( 5)   2
reporteldy/reportedly                   ( 5)   5
umcomplicated/uncomplicated             ( 5)   2
arimdex/arimidex                        ( 5)   4
aroound/around                          ( 5)   5
claoric/caloric                         ( 5)   1
incontinenece/incontinence              ( 5)   5
incrrease/increase                      ( 5)   5
iatrogentic/iatrogenic                  ( 5)   0
requirs/requires                        ( 5)   5
aprropriately/appropriately             ( 5)   5
popletial/popliteal                     ( 5)   0
dicarded/discarded                      ( 5)   1
respoding/responding                    ( 5)   5
pevlic/pelvic

## Show examples with scores

In [8]:
def textbf(text):
    return '\033[1m' + text + '\033[0m'

def textcolor(text, color):
    code = {'red': '31', 'green': '32', 'yellow': '33', 'blue': '34', 'magenta': '35', 'cyan': '36', 'white': '37'}
    return '\033[' + code[color] + 'm' + text + '\033[0m'

def print_results(example, outputss, model_names=None, correct_outputs=None, show_context=False, context_length=10, print_scores=False):
    typo, left, right, correct = example[-4:]
    print(' [Input]')
    print(f'typo / correct : {typo} / {correct}')
    if show_context:
        left = ' '.join(left.split()[-context_length:])
        right = ' '.join(right.split()[:context_length])
        print(f'Context: \"{left + " " + textcolor(textbf(typo), "red") + " " + right}\"')
    
    print(' [Output]')
    if not print_scores:
        for i, outputs in enumerate(outputss):
            outputs = [o[3:-4] for o in outputs['output']]
            output_string = ' '.join([textbf(o) if o == correct else o for o in outputs])
            rank = outputs.index(correct)+1 if correct in outputs else "X"
            if model_names:
                model_name = model_names[i]
            else:
                model_name = 'Output {i+1}'
            if rank == 1:
                model_color = 'green'
            elif rank == 'X':
                model_color = 'red'
            else:
                model_color = 'yellow'
            print(f'{textcolor(model_name, model_color) + "(" + str(rank) + ")":23s}: {output_string}')
    else:
        ranks = []
        for outputs in outputss:
            outputs = [o[3:-4] for o in outputs['output']]
            rank = outputs.index(correct)+1 if correct in outputs else "X"
            ranks.append(rank)
            
        first_line_texts = []
        for i, rank in enumerate(ranks):
            if model_names:
                model_name = model_names[i]
            else:
                model_name = 'Output {i+1}'
            if rank == 1:
                model_color = 'green'
            elif rank == 'X':
                model_color = 'red'
            else:
                model_color = 'yellow'
            first_line_texts.append(f'{textcolor(model_name, model_color) + "(" + str(rank) + ")":<35}' + "  Score      LM      ED")
        print(' | '.join(first_line_texts))
        
        for i in range(len(outputss[0]['output'])):
            texts = []
            for j in range(len(outputss)):
                o = outputss[j]['output'][i][3:-4]
                s = outputss[j]['scores'][i]
                l = outputss[j]['lm_scores'][i]
                e = s - l
                if o == correct:
                    o = f'{textbf(o):<33}'
                else:
                    o = f'{o:<25}'
                texts.append(o + f' {s:7.4f} {l:7.4f} {e:7.4f}')
            print(' | '.join(texts))
            
        texts = []
        for output in correct_outputs:
            assert output['correct'][3:-4] == correct
            s = output['score']
            l = output['lm_score']
            e = s - l
            texts.append(f'{"(Correct)":<25} {s:7.4f} {l:7.4f} {e:7.4f}')
        print(' | '.join(texts))

In [9]:
examples = clinspell_examples
resultss = [load_results(results_fpath_test_lmeddict)]
correct_resultss = [load_results(results_correct_fpath_test_lmeddict)]
model_names = ['LM+ED+dict']

def print_typo_results(typo, show_context=True, context_length=10, print_scores=False):
    correct = typo_correct_dict[typo]
    print(f'{typo}/{correct} ({total_cnt[typo]} total)')
    print(f'({typo} is {textcolor("in", "green") if typo in dictionary else textcolor("not in", "red")} dictionary)')
    print(f'({correct} is {textcolor("in", "green") if correct in dictionary else textcolor("not in", "red")} dictionary)')
    for name, cnt in zip(model_names, correct_cnt[typo]):
        print(f'- {name:10s}: {cnt} correct')
    print()

    for i in range(len(resultss[0])):
        ex_id = resultss[0][i]['example_id']
        example = examples[ex_id]
        outputss = [results[i] for results in resultss]
        correct_outputs = [results[i] for results in correct_resultss]
        if example[-4] == typo:
            print_results(example, outputss, model_names=model_names,
                          correct_outputs=correct_outputs, show_context=show_context, context_length=context_length, print_scores=print_scores)
            print('')

## 1. Easy typos - correct on all examples

In [10]:
print_typo_results('lugns', print_scores=True)

lugns/lungs (5 total)
(lugns is not in dictionary)
(lungs is in dictionary)
- LM+ED+dict: 5 correct

 [Input]
typo / correct : lugns / lungs
Context: "with hazy ground-glass opacity in the lower lobes of the lugns there is small amount of perihepatic ascites the evaluation of"
 [Output]
LM+ED+dict(1)               Score      LM      ED
lungs                     -1.1372 -0.3039 -0.8333
lung                      -2.8133 -0.8133 -2.0000
lings                     -2.8623 -1.1957 -1.6667
lines                     -3.0434 -0.5434 -2.5000
segments                  -3.4492 -0.6715 -2.7778
lunges                    -3.7488 -2.3202 -1.4286
lingula                   -3.7656 -0.6406 -3.1250
subsegments               -3.7657 -0.8490 -2.9167
liver                     -3.8211 -0.4877 -3.3333
subsegmental              -3.8220 -0.3604 -3.4615
substances                -3.8358 -0.6540 -3.1818
segment                   -3.9058 -0.7808 -3.1250
liker                     -3.9103 -0.5770 -3.3333
subsegment  

In [11]:
print_typo_results('responsvie', print_scores=True)

responsvie/responsive (5 total)
(responsvie is not in dictionary)
(responsive is in dictionary)
- LM+ED+dict: 5 correct

 [Input]
typo / correct : responsvie / responsive
Context: "murmur abd benign skin warm and well perfused alert and responsvie in nad"
 [Output]
LM+ED+dict(1)               Score      LM      ED
responsive                -0.8898 -0.4352 -0.4545
response                  -1.5967 -0.4856 -1.1111
responsives               -1.9633 -1.1300 -0.8333
responsively              -2.1855 -1.0317 -1.1538
responses                 -2.4262 -0.9262 -1.5000
responsiveness            -2.4830 -0.8164 -1.6667
responding                -2.5093 -0.6911 -1.8182
responsible               -2.5522 -1.3022 -1.2500
responsable               -2.5922 -1.3422 -1.2500
responsary                -2.6835 -1.3199 -1.3636
responsory                -2.7057 -1.3421 -1.3636
responsories              -2.8546 -1.7007 -1.1538
responsivity              -2.8852 -1.7314 -1.1538
responded                 -2.9828 

In [12]:
print_typo_results('containemnt', print_scores=True)

containemnt/containment (4 total)
(containemnt is not in dictionary)
(containment is in dictionary)
- LM+ED+dict: 4 correct

 [Input]
typo / correct : containemnt / containment
Context: "awake and irritable with cares settles with containemnt and pacifier brings hands to face temps stable in servo"
 [Output]
LM+ED+dict(1)               Score      LM      ED
containment               -1.5119 -1.0952 -0.4167
containments              -2.0672 -1.2980 -0.7692
continent                 -2.2486 -1.2486 -1.0000
continuing                -2.4927 -0.6745 -1.8182
continuant                -2.6507 -1.2870 -1.3636
confinement               -2.6567 -1.4067 -1.2500
continuous                -2.7353 -0.4625 -2.2727
containment's             -2.8935 -1.8221 -1.0714
continuation              -2.9695 -0.6618 -2.3077
contaminant               -2.9758 -1.7258 -1.2500
confinements              -3.0214 -1.4829 -1.5385
continence                -3.0547 -1.2365 -1.8182
continents                -3.0999 -1.736

In [13]:
print_typo_results('verrtex', print_scores=True)

verrtex/vertex (1 total)
(verrtex is not in dictionary)
(vertex is in dictionary)
- LM+ED+dict: 1 correct

 [Input]
typo / correct : verrtex / vertex
Context: "collection which herniates beyond the craniectomy defect notably along the verrtex superficial to this there is hyperdense appearance of the scalp"
 [Output]
LM+ED+dict(1)               Score      LM      ED
vertex                    -1.6319 -0.9176 -0.7143
vertebral                 -3.0844 -0.5844 -2.5000
vertebra                  -3.3293 -1.1070 -2.2222
ventral                   -3.4928 -0.9928 -2.5000
peritoneum                -3.5715 -0.8442 -2.7273
vertical                  -3.6090 -0.8312 -2.7778
peritoneal                -3.6241 -0.8968 -2.7273
vertebrae                 -3.7123 -1.2123 -2.5000
vertebro                  -3.7493 -1.5271 -2.2222
vertigo                   -3.7772 -1.2772 -2.5000
ventricle                 -3.7791 -0.7791 -3.0000
ventricles                -3.7867 -1.0594 -2.7273
extreme                   -3.872

## 2. Not easy typos - wrong on some examples

In [14]:
print_typo_results('claoric', print_scores=True)

claoric/caloric (5 total)
(claoric is not in dictionary)
(caloric is in dictionary)
- LM+ED+dict: 1 correct

 [Input]
typo / correct : claoric / caloric
Context: "lung excursion bilateral pleural effusions risk for altered nutrition poor claoric intake hours pain acute decreased gi motility gi inflammation"
 [Output]
LM+ED+dict(X)               Score      LM      ED
cardiac                   -3.2708 -0.7708 -2.5000
carotid                   -3.5173 -1.0173 -2.5000
clinical                  -3.6544 -0.8766 -2.7778
compliance                -3.7087 -0.5269 -3.1818
clinic                    -3.7362 -1.5934 -2.1429
carotic                   -3.8444 -1.9694 -1.8750
cardiogenic               -3.8455 -0.9289 -2.9167
complications             -4.0717 -0.5003 -3.5714
complication              -4.1080 -0.6464 -3.4615
contrast                  -4.1164 -0.7830 -3.3333
placement                 -4.1743 -0.6743 -3.5000
closely                   -4.2031 -1.0781 -3.1250
cardioid                  -4.21

In [15]:
print_typo_results('usally', print_scores=True)

usally/usually (5 total)
(usally is not in dictionary)
(usually is in dictionary)
- LM+ED+dict: 2 correct

 [Input]
typo / correct : usally / usually
Context: "pt alert and oriented times three obeying and following commands usally takes ativan tid in addition to klonopin mae in bed"
 [Output]
LM+ED+dict(1)               Score      LM      ED
usually                   -1.6973 -1.0723 -0.6250
unable                    -3.0931 -0.9503 -2.1429
slowly                    -3.5771 -0.7200 -2.8571
slightly                  -3.8513 -0.5180 -3.3333
safety                    -3.9474 -1.0902 -2.8571
usual                     -3.9866 -1.4866 -2.5000
unably                    -4.1734 -2.7449 -1.4286
halfway                   -4.2856 -1.1606 -3.1250
allergy                   -4.3321 -1.2071 -3.1250
saturation                -4.3489 -0.7126 -3.6364
sallowly                  -4.4681 -2.2459 -2.2222
split                     -4.4767 -1.1434 -3.3333
saline                    -4.4890 -1.6318 -2.8571
satur

In [16]:
print_typo_results('phelyephrine', print_scores=True)

phelyephrine/phenylephrine (3 total)
(phelyephrine is not in dictionary)
(phenylephrine is in dictionary)
- LM+ED+dict: 1 correct

 [Input]
typo / correct : phelyephrine / phenylephrine
Context: "and airway did not need intubation he was started on phelyephrine periperhally and transfer was requested to for liver transplant eval"
 [Output]
LM+ED+dict(X)               Score      LM      ED
perioperative             -3.4195 -0.5623 -2.8571
perioperatively           -3.7598 -0.6348 -3.1250
hypertensive              -3.7742 -0.6973 -3.0769
perinephric               -3.7883 -1.2883 -2.5000
hyperglycemia             -4.1778 -0.6064 -3.5714
elephantine               -4.2404 -1.7404 -2.5000
peripheral                -4.2429 -0.6065 -3.6364
hyperglycemic             -4.2869 -0.7155 -3.5714
hypertension              -4.2987 -0.8372 -3.4615
hyperinsulinemia          -4.3065 -1.0712 -3.2353
peripherals               -4.3371 -1.0037 -3.3333
hypertensively            -4.3972 -1.0639 -3.3333
hyperins

In [17]:
print_typo_results('nectotic', print_scores=True)

nectotic/necrotic (5 total)
(nectotic is not in dictionary)
(necrotic is in dictionary)
- LM+ED+dict: 2 correct

 [Input]
typo / correct : nectotic / necrotic
Context: "foot he was taken for guillotine bka on he had nectotic tissue at the base of his amputation site and was"
 [Output]
LM+ED+dict(1)               Score      LM      ED
necrotic                  -1.1322 -0.5766 -0.5556
ectopic                   -2.5275 -1.2775 -1.2500
necrosis                  -2.8891 -1.2224 -1.6667
necrotized                -3.0398 -1.2216 -1.8182
necrotizing               -3.0448 -0.9614 -2.0833
echogenic                 -3.3129 -0.8129 -2.5000
necrotically              -3.3461 -1.4231 -1.9231
negative                  -3.3649 -0.5872 -2.7778
necrotize                 -3.3822 -1.8822 -1.5000
necrosing                 -3.4286 -1.4286 -2.0000
necrotomic                -3.4958 -2.1322 -1.3636
necrotising               -3.6076 -1.5243 -2.0833
necrotica                 -3.6932 -2.6932 -1.0000
necrotizes    

## 3. Hard typos - wrong on all examples

In [18]:
print_typo_results('drainags', print_scores=True)

drainags/drainages (4 total)
(drainags is not in dictionary)
(drainages is in dictionary)
- LM+ED+dict: 0 correct

 [Input]
typo / correct : drainags / drainages
Context: "medical condition year old man with whipple with externalized biliary drainags dacron portal vein graft reason for this examination please assess"
 [Output]
LM+ED+dict(2)               Score      LM      ED
drainage                  -0.9653 -0.4098 -0.5556
drainages                 -2.0807 -1.5807 -0.5000
drains                    -2.1904 -0.7618 -1.4286
draining                  -2.3981 -0.7314 -1.6667
drain                     -2.8323 -0.3323 -2.5000
drained                   -3.1669 -1.2919 -1.8750
diagnosis                 -3.6143 -1.1143 -2.5000
drainers                  -3.6261 -2.5150 -1.1111
drainable                 -3.6906 -2.1906 -1.5000
dialysis                  -3.7000 -0.9222 -2.7778
transplant                -3.7514 -0.5695 -3.1818
transgress                -3.8064 -1.0791 -2.7273
diagnoses            

In [19]:
print_typo_results('contonese', print_scores=True)

contonese/cantonese (5 total)
(contonese is not in dictionary)
(cantonese is in dictionary)
- LM+ED+dict: 0 correct

 [Input]
typo / correct : contonese / cantonese
Context: "be taken specifically for cancer at present response ongoing plan contonese interpreter as needed family aware of transfer to hospital ward"
 [Output]
LM+ED+dict(X)               Score      LM      ED
continue                  -2.0261 -0.3594 -1.6667
continues                 -2.5571 -1.0571 -1.5000
continuous                -2.7303 -0.4576 -2.2727
continuously              -2.8426 -0.5350 -2.3077
continence                -2.9606 -1.5969 -1.3636
continent                 -2.9637 -1.4637 -1.5000
continuing                -2.9988 -0.7261 -2.2727
continued                 -3.0193 -1.0193 -2.0000
continous                 -3.1776 -1.1776 -2.0000
concentrate               -3.2609 -0.7609 -2.5000
constantly                -3.2817 -1.0089 -2.2727
continuate                -3.2997 -1.4816 -1.8182
continual               

In [20]:
print_typo_results('hypercholesteromia', print_scores=True)

hypercholesteromia/hypercholesteremia (5 total)
(hypercholesteromia is not in dictionary)
(hypercholesteremia is in dictionary)
- LM+ED+dict: 0 correct

 [Input]
typo / correct : hypercholesteromia / hypercholesteremia
Context: "brief hospital course patient is yo man with pmh hypercholesteromia presented to with uri and sob found to have nstemi"
 [Output]
LM+ED+dict(2)               Score      LM      ED
hypercholesterolemia      -0.8211 -0.3449 -0.4762
hypercholesteremia        -1.0621 -0.7990 -0.2632
hypercholesterolia        -1.3052 -1.0420 -0.2632
hypercholesterolemic      -1.4512 -0.7369 -0.7143
hypercholesteremic        -1.6253 -1.0989 -0.5263
hypercholesterolemias     -1.6295 -0.9477 -0.6818
hypercholesteremias       -1.6399 -1.1399 -0.5000
hypercholesterol          -1.6705 -0.7881 -0.8824
hypercholesterolaemia     -1.9424 -1.2606 -0.6818
hypercholesteraemia       -2.0063 -1.5063 -0.5000
hypercholesterinemia      -2.1300 -1.4157 -0.7143
hypercholesterolinemia    -2.1670 -1.2974

In [21]:
print_typo_results('eppiplocae', print_scores=True)

eppiplocae/epiploicae (1 total)
(eppiplocae is not in dictionary)
(epiploicae is in dictionary)
- LM+ED+dict: 0 correct

 [Input]
typo / correct : eppiplocae / epiploicae
Context: "ecchymotic with significant retroperitoneal edema and bowel edema the appendages eppiplocae were hemorrhagic there was pathc purpuring darkening concerning for ischemia"
 [Output]
LM+ED+dict(13)              Score      LM      ED
epiploic                  -3.9146 -1.6924 -2.2222
episode                   -4.3165 -1.1915 -3.1250
episodically              -4.3489 -1.2720 -3.0769
episodes                  -4.4022 -1.0689 -3.3333
epicardial                -4.4629 -1.2811 -3.1818
epidermal                 -4.5479 -1.5479 -3.0000
epicardially              -4.5862 -1.1247 -3.4615
epidermically             -4.6131 -1.3988 -3.2143
epidermical               -4.6795 -1.7629 -2.9167
epiplocele                -4.6887 -3.3251 -1.3636
epidermoids               -4.6913 -1.3579 -3.3333
epiploitis                -4.7241 -2.45